Assignment 1 Julianne Lin, Brandon Wada



In [1]:
# We put these at the top
%reload_ext autoreload
%autoreload 2
%matplotlib inline

We used the following commands once to download the data.

%env KAGGLE_USERNAME=hmcclyde
%env KAGGLE_KEY=f8be01b52e62422dcb2e983b72ec1300
!kaggle competitions download -c dog-breed-identification

We manually moved the files from the notebooks folder that they were downloaded into to the data folder using Jupyter lab's file explorer

----
One time unzip the folders into our directory

!unzip ../../data/dogbreeds/labels.csv.zip -d ../../data/dogbreeds
!unzip ../../data/dogbreeds/test.zip -d ../../data/dogbreeds
!unzip ../../data/dogbreeds/train.zip -d ../../data/dogbreeds

In [ ]:
#One time use to create a directory for each dog breed
"""
import csv
import os

#make a validation folder
#os.makedirs("../../data/dogbreeds/valid")

#iterate through the csv file to collect all the dog breeds
breeds = []
with open('../../data/dogbreeds/labels.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if not ((row['breed']) in breeds):
            L.append(row['breed'])
            
mkdirtrainpath = "../../data/dogbreeds/train/"
mkdirvalidpath = "../../data/dogbreeds/valid/"
for i in L:
    os.makedirs(mkdirtrainpath + i)
    os.makedirs(mkdirvalidpath + i)
"""


In [2]:
#One time use to sort the data into the respective folder
"""
import shutil
import csv
folder  = "../../data/dogbreeds/train/"
with open('../../data/dogbreeds/labels.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        shutil.move(folder + row['id'] + ".jpg",folder + row['breed'])
"""

In [17]:
#One time use to create validation set
"""
import random
trainingpath = "../../data/dogbreeds/train/"
breedfolders = os.listdir(trainingpath)
for i in breedfolders:
    for j in range(6):
        pics = os.listdir(trainingpath + i)
        shutil.move(trainingpath + i + "/" + (random.choice(pics)),"../../data/dogbreeds/valid/" + i)
"""

Having created the necessary directory structure, we now can build our net

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

/home/bwada/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


sets variables, using resnet34 pretrained model.

In [3]:
arch=resnet34
PATH = "../../data/dogbreeds/"
sz=224

Sets the data and hyperparameters

In [18]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz),
                                      test_name="test")

NOW WE LEARN

In [19]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 2)

100%|██████████| 162/162 [00:58<00:00,  2.57it/s]


epoch      trn_loss   val_loss   accuracy   
    0      1.843428   0.97659    0.768056  
    1      0.953113   0.67958    0.811111  



[array([0.67958]), 0.8111111111111111]

Having trained the net, we can make predictions

In [20]:
log_preds, y = learn.predict_with_targs(is_test=True) # use test dataset rather than validation dataset
probs = np.exp(log_preds)

We then fill a pandas datafram

In [22]:
probs.shape # (n_images, n_classes)

(10357, 120)

In [24]:
data.classes

In [30]:
probs[:,:]

array([[0.00038, 0.00177, 0.00178, ..., 0.0461 , 0.00134, 0.00175],
       [0.00084, 0.00001, 0.00042, ..., 0.00009, 0.00003, 0.00007],
       [0.00012, 0.00011, 0.00004, ..., 0.00011, 0.00012, 0.00004],
       ...,
       [0.0001 , 0.00001, 0.00004, ..., 0.00012, 0.28986, 0.00006],
       [0.00041, 0.00005, 0.00027, ..., 0.00112, 0.0002 , 0.00073],
       [0.00371, 0.00001, 0.00001, ..., 0.00001, 0.     , 0.00649]], dtype=float32)

In [40]:
df = pd.DataFrame(probs[:,:])
df.columns = [data.classes]

In [41]:
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [43]:
df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,f1705303b30da3a48db2a5f34376b947,0.000385,0.001774,0.001778,0.008977,0.004484,0.000376,0.000681,0.000615,0.000210,...,0.001249,0.000963,0.156417,0.001613,0.372945,0.000278,0.000131,0.046104,0.001335,0.001748
1,a9b79809bc1301f8d148e15bfecf6485,0.000843,0.000008,0.000423,0.000009,0.000111,0.001837,0.000243,0.000077,0.000098,...,0.000202,0.000407,0.000033,0.000493,0.000228,0.000092,0.205219,0.000089,0.000033,0.000074
2,cc94a3bb7191a247f59e1540d8430c93,0.000121,0.000107,0.000044,0.000024,0.000077,0.092876,0.000018,0.000090,0.000301,...,0.000024,0.000069,0.000080,0.000264,0.000011,0.001401,0.000082,0.000112,0.000120,0.000038
3,f7b489a38302afc010d8a386a1b7d085,0.000023,0.000115,0.000007,0.000010,0.000022,0.144174,0.000004,0.000060,0.000150,...,0.000004,0.000025,0.000004,0.000147,0.000005,0.000264,0.000019,0.000043,0.000023,0.000009
4,6e21c96226078576342ca66e25eb1aa9,0.007975,0.001362,0.000380,0.002722,0.000114,0.000352,0.000223,0.000524,0.000638,...,0.000168,0.000097,0.001033,0.000238,0.000414,0.000103,0.000550,0.000420,0.002597,0.000191


In [45]:
SUBM = f'../../data/dogbreeds/'
df.to_csv(f'{SUBM}dogsbreeds_JLinBWada.gz', compression='gzip', index=False)

We then submit the csv file for the test data to kaggle

<img src="images/NNHW1.png" width="80%">